### Assignment 1 (due Wednesday, Sep 10th, 11:59 pm)

Submit your completed .ipynb file to the corresponding Brightspace assignment.

#### Overview: 

In this assignment, you’ll become familiar with PyTorch by following the [official tutorial](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html). You will also implement a classifier for the CIFAR-10 dataset and adapt it to use the d2l library’s API.


#### Tasks: 
* **Follow the tutorial:** Start by reading and completing the tutorial to gain an understanding of PyTorch basics.
* **Implement a CIFAR-10 Classifier:** Build a classifier for the CIFAR-10 dataset. You are free to design the network structure and choose the hyperparameters.
* **Port to d2l API:** Adapt your classifier to the d2l library by creating two classes: ***d2l.Classifier*** and ***d2l.DataModule***. Use these to fit your model.

#### Hints:
* For the d2l API implementation, you can refer to examples from previous chapters (e.g., Example 5.6). Chapters 6 and 23.8 are also good references.
* You can check the source code in d2l, such as the FashionMNIST dataset implementation in `d2l/torch.py`.
* Your final model should achieve a **validation accuracy (val_acc) of at least 0.70**, similar to the example figure below.
<br><br>


<div align="center">
  <img src="attachment:a34e5eb8-fecb-4d52-ab9d-da1286186b84.png" alt="image.png">
</div>

## Your d2l.Classifier and d2l.DataModule

In [1]:
##### YOUR CLASSIFIER AND DATAMODULE HERE #####



## Training Routine
Your training routine should train the classifier and output a loss plot similar to the one above, where the validation accuracy is at least 0.7 at the end of the run.   Please use functions from the d2l library to train (the Trainer class, .fit()).

In [2]:
##### YOUR TRAINING ROUTINE HERE #####

